# 排序滤波

In [27]:
#coding:utf-8
%matplotlib inline
import numpy as np
import scipy
from scipy.special import comb
import matplotlib.pyplot as plt
import cv2
from scipy import stats
import scipy.ndimage.filters as nd_filters
import time

In [28]:
gray = cv2.imread('../../datas/f4.jpg',0)
rgb = cv2.cvtColor(cv2.imread('../../datas/f4.jpg'),cv2.COLOR_BGR2RGB)
rows,cols,ch = rgb.shape

In [29]:
def noise_gaussian(M,N,a = 0,b=1):
    R = a + b * np.random.randn(M,N)
    return R

In [30]:
noise = noise_gaussian(rows,cols,a=0,b=0.1)
src_noise_gray = gray.astype(np.float32) + noise * 255
src_noise_gray[src_noise_gray < 0] = 0
src_noise_gray[src_noise_gray > 255] = 255
src_noise_gray = src_noise_gray.astype(np.uint8)

In [31]:
def add_syn_border(u,padsize,Xind,Yind):
    # add_border of width 5 to get an output image of size [256,256] 

    m, n = Xind.shape
    p1, p2 = padsize

    v = zeros((m, n))
    v[p1 : m-p1, p2 : n-p2] = u
    for i in r_[xrange(p1), xrange(m-p1, m)]:
        for j in xrange(n):
            v[i, j] = v[Xind[i, j],Yind[i, j]]
    for i in xrange(p1, m-p1):
        for j in r_[xrange(p2), xrange(n-p2, n)]:
            v[i, j] = v[Xind[i, j],Yind[i, j]]
    return v

In [32]:
def padarray(A, padsize, method='zero', direction='both',
      Xind=None, Yind=None):
    """
    pads array A using the specified method and direction. 
    method can be one of these strings: 
       'zero'           Pads with zeros
       'perodic'        Pads with circular repetitiion of elements
       'symmetric'      Pads array with mirror reflections of itself 
       'antisymmetric'  Pads array with point reflections of itself 
       'synthteic'      Pads array with synthetic boundary conditions
    direction can be one of the following strings.  
       'pre'            Pads before the first array element along each
                        dimension .
       'post'           Pads after the last array element along each
                        dimension. 
       'both'           Pads before the first array element and after the
                        last array element
    Example:
    >>> from numpy import arange
    >>> from padarray import padarray
    >>> n = 5
    >>> A = arange(n ** 2)
    >>> A.shape = (n, n)
    >>> padsize = (2,3)
    >>> B = padarray(A,padsize,'periodic','both')
    >>> B
    array([[17, 18, 19, 15, 16, 17, 18, 19, 15, 16, 17],
           [22, 23, 24, 20, 21, 22, 23, 24, 20, 21, 22],
           [ 2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2],
           [ 7,  8,  9,  5,  6,  7,  8,  9,  5,  6,  7],
           [12, 13, 14, 10, 11, 12, 13, 14, 10, 11, 12],
           [17, 18, 19, 15, 16, 17, 18, 19, 15, 16, 17],
           [22, 23, 24, 20, 21, 22, 23, 24, 20, 21, 22],
           [ 2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2],
           [ 7,  8,  9,  5,  6,  7,  8,  9,  5,  6,  7]])
    """

    a0, a1 = A.shape
    p0, p1 = padsize

    if method == 'zero':
        if direction == 'pre':
            B = np.zeros((a0 + p0, a1 + p1))
            B[p0 : p0 + a0, p1 : p1 + a1] = A          
        elif direction == 'post':
            B = np.zeros((a0 + p0, a1 + p1))
            B[0 : a0, 0 : a1] = A          
        elif direction == 'both':
            B = np.zeros((a0 + 2*p0, a1 + 2*p1))
            B[p0 : p0 + a0, p1 : p1 + a1] = A          

    elif method == 'periodic':
        if direction == 'pre':
            B = A[ix_(range(-p0, a0), range(-p1, a1))]
        elif direction == 'post':
            B = A[ix_(mod(range(0,a0 + p0), a0), mod(range(0,a1 + p1), a1))]
        elif direction == 'both':
            B = A[ix_(mod(range(-p0,a0+p0), a0), mod(range(-p1,a1+p1), a1))]

    elif method == 'reflexive':
        if direction == 'pre':
            B = A[ix_(range(p0-1, -1, -1) + range(0, a0),
                      range(p1-1, -1, -1) + range(0, a1))]
        elif direction == 'post':
            B = A[ix_(range(0, a0) + range(a0-1, a0-p0-1, -1),
                      range(0, a1) + range(a1-1, a1-p1-1, -1))]
        elif direction == 'both':
            B = A[ix_(range(p0-1,-1,-1)+range(0,a0)+range(a0-1,a0-p0-1,-1),
                      range(p1-1,-1,-1)+range(0,a1)+range(a1-1,a1-p1-1,-1))]
    
    elif method == 'antireflexive':
        if direction == 'pre':
            B0 = A[ix_([0]*p0 + range(0, a0),
                       [0]*p1 + range(0, a1))]
            B1 = A[ix_(range(p0, 0, -1) + range(0, a0),
                       range(p1, 0, -1) + range(0, a1))]
        elif direction == 'post':
            B0 = A[ix_(range(0, a0) + [a0-1]*p0,
                       range(0, a1) + [a1-1]*p1)]
            B1 = A[ix_(range(0, a0) + range(a0-2, a0-p0-2, -1),
                       range(0, a1) + range(a1-2, a1-p1-2, -1))]
        elif direction == 'both':
            B0 = A[ix_([0]*p0 + range(0, a0) + [a0-1]*p0,
                       [0]*p1 + range(0, a1) + [a1-1]*p1)]
            B1 = A[ix_(range(p0,0,-1)+range(0,a0)+range(a0-2,a0-p0-2,-1),
                       range(p1,0,-1)+range(0,a1)+range(a1-2,a1-p1-2,-1))]
        B = 2 * B0 - B1
    elif method == 'synthetic':
        B = add_syn_border(A, padsize, Xind, Yind)
    return B 

In [33]:
def order_filter(img,m = 3):
    n = (m - 1) // 2
    M,N = img.shape[0],img.shape[1]
    tempPic = np.zeros((M + n * 2,N + n * 2),dtype = img.dtype)
    min_pic = np.zeros_like(img)
    mid_pic = np.zeros_like(img)
    max_pic = np.zeros_like(img)
    sroted_pic = np.zeros((m*m,))
    
    for i in range(n+1,M+n):
        for j in range(n+1,N + n):
            tempMatrix = img[(i-n):(i+n),(j-n):(j+n)].tolist()
            sorted_pic = np.sort(tempMatrix)
            min_pic[(i-n):(i+n),(j-n):(j+n)] = sorted_pic[0]
            max_pic[(i-n):(i+n),(j-n):(j+n)] = sorted_pic[-1]
            mid_pic[(i-n):(i+n),(j-n):(j+n)] = sorted_pic[(m*m+1) // 2]
            
    return min_pic,mid_pic,max_pic

In [34]:
min_pic,mid_pic,max_pic = order_filter(gray)

IndexError: index 5 is out of bounds for axis 0 with size 2